In [1]:
import pandas as pd

In [26]:
import random
import re
import time

In [2]:
df =  pd.read_csv('precios.csv')

In [9]:
df['Dependencia'][df['Dependencia'].str.startswith("Error")].value_counts()

Dependencia
Error::: Descripción detallada,Unidad de medida,Cantidad mínima,Cantidad máxima,Precio unitario sin impuestos,Monto total cantidad mínima,Monto total cantidad máxima    296
Error::: Descripción detallada,Unidad de medida,Cantidad mínima,Cantidad máxima,Precio unitario sin impuestos,Monto total cantidad mínima,                               209
Error::: Descripción detallada,Unidad de medida,Precio unitario sin impuestos,Monto de la Oferta,IVA,Otros impuestos,Monto total de la oferta                            180
Error::: Descripción detallada,Unidad de medida,Precio unitario sin impuestos,Monto de la Oferta,IVA,,                                                                   100
Error::: Descripción detallada,Unidad de medida,Cantidad mínima,Cantidad máxima,,,                                                                                        30
Name: count, dtype: int64

In [7]:
df['Dependencia'].shape

(5692,)

In [5]:
procedimientos_guardados = pd.read_csv('concluidos.csv', usecols = ['num_proc'])

In [7]:
if procedimientos_guardados['num_proc'].str.contains('').any():
            print("ya está en la bdd")

ya está en la bdd


In [7]:
df = pd.read_excel('./Claves de partida.xlsx', sheet_name='Hoja1', header=None)

In [13]:
claves = df[5].unique()

In [18]:
claves = ['15901', '26101', '26102', '26103', '15401', '21101', '37501', '37502', '37503', 
              '37504', '37601', '37602', '37701', '37801', '37802', '37901', '38101', '38102', 
              '38103', '38201', '38301', '38401', '38501', '44101', '44102', '44103', '44104', 
              '34101']

In [20]:
print(random.choice(claves))

37801


In [21]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import TimeoutException, ElementClickInterceptedException
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from datetime import date, datetime

In [22]:
#Se metio un error debido a la deshabilitación de cookies de 3ros en chrome.
executable_path = ChromeDriverManager().install()
executable_path = executable_path.replace("THIRD_PARTY_NOTICES.chromedriver", "chromedriver.exe")

#Dispara chrome
driver = webdriver.Chrome(
            service=Service(executable_path=executable_path),
        )
print("driver correctamente inicializado.")

driver correctamente inicializado.


In [23]:
driver.get('https://upcp-compranet.hacienda.gob.mx/sitiopublico/#/sitiopublico/detalle/b005f29806134df89466ca3c04d8a392/proyecto')

In [24]:
#Esto debido a que a veces economicos no cargaba, y aveces trae un layout distinto
xp_1 = '//th[text()="Descripción CUCoP+"]/ancestor::div[contains(@class,"header")]/following-sibling::div/table//tr'
xp_2 = '//th[text()="Descripción CUCoP+"]/ancestor::thead/following-sibling::tbody/tr'
xp_economicos = xp_1
economicos = driver.find_elements(By.XPATH, xp_1)
iterations = 0
if len(economicos)<1:
    print('  No cargaron economicos, se esperará')
    while True:
        economicos = driver.find_elements(By.XPATH, xp_2)
        if len(economicos)>0:
            xp_economicos = xp_2
            print(f"   economicos tipo 2")
            break

        economicos = driver.find_elements(By.XPATH, xp_1)
        if len(economicos)>0:
            xp_economicos = xp_1
            break
        
        iterations += 1
        if iterations>10:
            print('   No cargaron.')
            break

In [59]:
economicos_list = []
economic_ids = []
#Itera páginas
for i in range(1, 11):
    print(f"  Obteniendo economicos pag: {i}")
    economicos = driver.find_elements(By.XPATH, xp_economicos)
    numero = economicos[0].text.split(" ",1)[0]
    #Itera filas de economicos
    for row_economic in economicos:
        col = row_economic.find_elements(By.XPATH,"./td")
        _id = col[0].text
        if _id in economic_ids:
            print(f"economico {_id} está repetido por alguna extraña razón")
        partida_esp = col[1].text
        clave_cucop = col[2].text
        desc_cucop = col[3].text
        desc_det = col[4].text
        unidades_medida = col[5].text
        #Esta columna algunas pocas veces no viene
        cantidad_solicitada = col[6].text if len(col)>6 else ""
        #Busca un patron de 3 o 4 digitos, punto, 3 o 4 digitos, punto,3 o 4 digitos, y desues otros 2 digitos opcionales
        claves_compendio = re.findall(r"\d{3,4}.{1}\d{3,4}.{1}\d{3,4}.?\d{0,2}", desc_cucop)#CUCOP
        clave_compendio = claves_compendio[0].strip().replace(" ",".") if len(claves_compendio)>0 else ""

        economicos_list.append({
            "Clave compendio":clave_compendio, "Codigo del expediente":'cod_exp', "Número del procedimiento o contratación":'num_proc',
            "Partida específica":partida_esp, "Clave CUCoP+":clave_cucop, "Descripción CUCoP+": desc_cucop, 
            "Descripción detallada":desc_det, "Unidad de medida":unidades_medida, "Cantidad solicitada":cantidad_solicitada,
            "uri":'uri', "scrapped_day":'today'
        })
        economic_ids.append(_id)

    next_page_btn = driver.find_element(By.XPATH,'//span[text()="ECONÓMICOS"]/ancestor::p-tabview//button[contains(@class,"p-paginator-next")]')
    if "p-disabled" in next_page_btn.get_attribute("class"):
        print("  no hay más páginas disponibles")
        break

    for i in range(0, 10):
        #Revisa si hay más paginas disponibles
        next_page_btn = driver.find_element(By.XPATH,'//span[text()="ECONÓMICOS"]/ancestor::p-tabview//button[contains(@class,"p-paginator-next")]')
        if "p-disabled" in next_page_btn.get_attribute("class"):
            print("  no hay más páginas disponibles")
            break

        #Da click en next page
        ActionChains(driver).scroll_to_element(next_page_btn)\
            .pause(1).click(next_page_btn).perform()
        
        time.sleep(5)

        #Espera nuevos economicos por un máximo de 60 segundos 
        t_end = time.time() + 60
        numero_nuevo = driver.find_elements(By.XPATH, xp_economicos)[0].text.split(" ",1)[0]
        while time.time() < t_end and numero == numero_nuevo:
            try:
                numero_nuevo = driver.find_elements(By.XPATH, xp_economicos)[0].text.split(" ",1)[0]
            except:
                pass
        
        #Sí aparecieron nuevos economicos los recolecta...
        if int(numero_nuevo) > int(numero):
            break
        else:# sí no aparecieron nuevos economicos en un minuto busca la siguiente página.
            print("  no aparecieron nuevos economicos en un minuto")
            

  Obteniendo economicos pag: 1
  Obteniendo economicos pag: 2
  Obteniendo economicos pag: 3
  Obteniendo economicos pag: 4
  Obteniendo economicos pag: 5
  Obteniendo economicos pag: 6
  Obteniendo economicos pag: 7
  Obteniendo economicos pag: 8
  no hay más páginas disponibles


In [60]:
from colorama import Fore

In [71]:
print(Fore.RED+ 'i'+Fore.RESET)
print('i')
print('i')
print('i')
print('i')
print(Fore.YELLOW+ 'i')
print('i')
print('i')

i
i
i
i
i
i
i
i


In [65]:
for i in range (0, 15):
    if i>10:
        print(Fore.RESET, i)
    else:
        print(Fore.RED, i)

 0
 1
 2
 3
 4
 5
 6
 7
 8
 9
 10
 11
 12
 13
 14


In [61]:
len(economicos_list)

394

In [72]:
driver.quit()